In [96]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import functools
import scipy.signal
import tsfresh
import pickle
from tqdm import tqdm
import inspect
import sys
import json
from scipy.signal import savgol_filter
sys.path.append("src/data_processing/")
sys.path.append("src/validation/")
import dp_utils as dp

In [ ]:
%%time

filepath = '/home/sergey/Projects/Kaggle/LANL-Earthquake-Prediction/train.csv'

df = pd.read_csv(filepath, 
                 dtype={"acoustic_data": np.int16, "time_to_failure": np.float32}, 
                 skiprows=1, 
                 names=['s', 'ttf'])

In [ ]:
filepath = '/home/sergey/Projects/Kaggle/LANL-Earthquake-Prediction/train/train.h5'

df.to_hdf(filepath, key='table')

In [5]:
filepath = '/home/sergey/Projects/Kaggle/LANL-Earthquake-Prediction/train/train.h5'
# filepath = '/Users/sergey/Dev/Kaggle/LANL-Earthquake-Prediction/train/train.h5'
# filepath = '/Users/sergey/Dev/Kaggle/LANL-Earthquake-Prediction/train/train_short.h5'

df = pd.read_hdf(filepath, key='table')

In [3]:
# filepath = './data/train_short_processed.h5'
filepath = '/Users/sergey/Dev/Kaggle/LANL-Earthquake-Prediction/train/train_processed.h5'

df_processed = pd.read_hdf(filepath, key='table')

In [ ]:
df_processed


In [ ]:
from inspect import getmembers, isfunction

funcname_list = [o[0] for o in getmembers(dp) if isfunction(o[1])]
funcref_list = [o[1] for o in getmembers(dp) if isfunction(o[1])]

In [ ]:
import json

dp_config = {"data_path": "../../data/train_short.h5", 
             "data_processed_path": "../../data/train_short_processed.h5", 
             "window_length": 10000,
             "routines": {}}

In [ ]:
for obj in funcref_list[:-1]:
    inspect_obj = inspect.signature(obj)
    params_dict = dict(inspect_obj.parameters)
    params = {}
    for k, v in params_dict.items():
        if v.default != inspect._empty:
            params[k] = v.default      
    dp_config["routines"][obj.__name__] = {"on": True, "params": params}
    
dp_config

In [ ]:
func_list = []
for name in list(dp_config['routines'].keys()):
    func_list.append(getattr(dp, name))

# 3. Run processing
routine_settings = list(dp_config['routines'].values())

dfp = pd.concat(
    [func(df['s'], **setting['params']) for func, setting in zip(func_list, routine_settings) if setting['on']],
    axis=1)

In [4]:
with open("/Users/sergey/Documents/Projects/Kaggle/LANL-Earthquake/src/validation/Model 1", 'rb') as file:  
    pickle_model = pickle.load(file)

/anaconda3/envs/p-sci/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [ ]:
plt.figure(figsize=(10,5))
plt.plot(pickle_model.predict(df_processed.drop('w_labels', axis=1)), 'k')
plt.plot(df_processed['w_labels'], 'r')

In [ ]:
abs(pickle_model.predict(df_processed.drop('w_labels', axis=1)) - df_processed['w_labels']))

In [97]:
class WindowDecorator(object):
    def __init__(self, f=None):
        self.f = f
        with open("dp_config.json") as config:
            params = json.load(config)
        self.window_size = params['window_size']

    def __call__(self, *args, **kwargs):
        if self.f is None:
            print(args)
            self.f = args[0]
        temp = []
        df = args[0]
        inspect_params = inspect.getfullargspec(self.f)
        for i in tqdm(range(0, df.shape[0], self.window_size), 
                desc=self.f.__name__ + "({}, ".format(*inspect_params.args)
                      + ', '.join("{}={})".format(k, v) for k, v in kwargs.items()), file=sys.stdout):
            batch = df.iloc[i: i+self.window_size].values
            temp.append(self.f(batch, *args, **kwargs))
        tqdm.write("\t window decorator: ")
        tqdm.write("\t - window size: {}".format(self.window_size))
        return pd.DataFrame(temp, columns={self.f.__name__})

In [98]:
@WindowDecorator
def w_mean(df, *args, **kwargs):
    return np.mean(df)

@WindowDecorator
def w_min(df, *args, **kwargs):
    return np.mean(df)


In [99]:
w_mean.window_size = 100000
w_mean(df['s'])
w_min(df['s'])

w_mean(df, : 100%|██████████| 6292/6292 [00:00<00:00, 7413.09it/s]
	 window decorator: 
	 - window size: 100000
w_min(df, : 100%|██████████| 62915/62915 [00:04<00:00, 13211.21it/s]
	 window decorator: 
	 - window size: 10000


,w_min
0,5.182000
1,4.917400
2,4.935700
3,4.961500
4,4.813900
5,4.764700
6,5.152700
7,5.182300
8,5.061300
9,5.187900


In [84]:
test(df['s'])

<function __main__.w_mean(df, *args, **kwargs)>

In [41]:
w_mean(df['s'])

w_mean(df, : 100%|██████████| 6292/6292 [00:00<00:00, 7205.79it/s]
	 window decorator: 
	 - window size: 100000


,w_mean
0,5.015940
1,4.659430
2,4.739450
3,4.815880
4,4.965530
5,4.931540
6,4.948550
7,4.949920
8,4.834880
9,4.901360


In [25]:
def filter_decorator(window=21, poly_order=1):
    def filter_calc(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            df = func(args[0], *args, **kwargs)
            temp = savgol_filter(df.squeeze(), window, poly_order)
            return pd.DataFrame(temp, columns={func.__name__})
        return wrapper
    return filter_calc
        
def function_decorator(f, params):
    def filter_calc(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            df = func(args[0], *args, **kwargs)
            temp = f(df.squeeze(), **params)
            tqdm.write("\t function decorator: ")
            tqdm.write("\t - {}{}".format(f.__name__, inspect.signature(f)))
            tqdm.write("\t - params: {}".format(params))
            return pd.DataFrame(temp, columns={func.__name__ + '+' + f.__name__ + '({})'.format(params)})
        return wrapper
    return filter_calc

def window_decorator(window_size=10000):
    def window_calc(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            temp = []
            df = args[0]
            inspect_params = inspect.getfullargspec(func)
            for i in tqdm(range(0, df.shape[0], window_size), 
                    desc=func.__name__ + "({}, ".format(*inspect_params.args)
                          + ', '.join("{}={})".format(k, v) for k, v in kwargs.items()), file=sys.stdout):
                batch = df.iloc[i: i+window_size].values
                temp.append(func(batch, *args, **kwargs))
            tqdm.write("\t window decorator: ")
            tqdm.write("\t - window size: {}".format(window_size))
            return pd.DataFrame(temp, columns={func.__name__})
        return wrapper
    return window_calc

In [26]:
@function_decorator(savgol_filter, {"window_length": 21, "polyorder": 1})
@window_decorator()
def w_mean(df, *args, **kwargs):
    return np.mean(df)

@function_decorator(savgol_filter, {"window_length": 21, "polyorder": 1})
@window_decorator()
def w_min(df, *args, **kwargs):
    return np.min(df)

@filter_decorator()
@window_decorator()
def w_psd(df, *args, fs=4e6, **kwargs):
    return np.sum(scipy.signal.periodogram(df, fs=fs)[1])

@window_decorator()
def w_psd_raw(df, *args, fs=4e6, **kwargs):
    return np.sum(scipy.signal.periodogram(df, fs=fs)[1])

In [27]:
w_mean(df['s'])
w_min(df['s'])

w_mean(df, : 100%|██████████| 100/100 [00:00<00:00, 6012.13it/s]
	 window decorator: 
	 - window size: 10000
	 function decorator: 
	 - savgol_filter(x, window_length, polyorder, deriv=0, delta=1.0, axis=-1, mode='interp', cval=0.0)
	 - params: {'window_length': 21, 'polyorder': 1}
w_min(df, : 100%|██████████| 100/100 [00:00<00:00, 7905.13it/s]
	 window decorator: 
	 - window size: 10000
	 function decorator: 
	 - savgol_filter(x, window_length, polyorder, deriv=0, delta=1.0, axis=-1, mode='interp', cval=0.0)
	 - params: {'window_length': 21, 'polyorder': 1}


/anaconda3/envs/p-sci/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


,"w_min+savgol_filter({'window_length': 21, 'polyorder': 1})"
0,-25.839827
1,-27.289177
2,-28.738528
3,-30.187879
4,-31.637229
5,-33.086580
6,-34.535931
7,-35.985281
8,-37.434632
9,-38.883983


In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(w_psd_raw(df['s']), 'k')
plt.plot(w_psd(df['s']), 'r')

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(w_mean(df['s']).values, 'k')
plt.plot(savgol_filter(w_mean(df['s']).values.squeeze(), window_length=21, polyorder=2), 'r')
plt.grid(True)
plt.show()

In [ ]:
predictions = pickle_model.predict(df_processed.drop(['w_labels'], axis=1))
actual_values = df_processed['w_labels']
print(len(abs(predictions - actual_values)))

plt.figure(figsize=(20, 10))
# plt.plot(predictions, 'r')
plt.plot(scipy.signal.periodogram(actual_values.values, fs=4e2)[1], 'k')
plt.ylim([0, 100])
plt.xlim([0, 1000])
plt.grid(True)
plt.show()

In [ ]:
# Import the necessary modules and libraries
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

# Create a random dataset
rng = np.random.RandomState(1)
X = np.sort(15 * rng.rand(800, 1), axis=0)
y = np.sin(X).ravel()
# y[::5] += 3 * (0.5 - rng.rand(16))

# Fit regression model
regr_1 = DecisionTreeRegressor(max_depth=1)
regr_2 = DecisionTreeRegressor(max_depth=3)
regr_1.fit(X, y)
regr_2.fit(X, y)

# Predict
X_test = np.arange(0.0, 15.0, 0.01)[:, np.newaxis]
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)

# Plot the results
plt.figure()
plt.scatter(X, y, s=20, edgecolor="black",
            c="darkorange", label="data")
plt.plot(X_test, y_1, color="cornflowerblue",
         label="max_depth=2", linewidth=2)
plt.plot(X_test, y_2, color="yellowgreen", label="max_depth=5", linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()

In [ ]:
df['s'].values

In [ ]:
t = {}
if not t:
    print("H")

In [ ]:
@window_decorator()
def w_last(df, *args, **kwargs):
    return df[-1]

In [ ]:
df_processed = df_processed.join(dp.w_labels(df['ttf']))

In [ ]:
df_processed

In [ ]:
w_last(df['ttf'])

In [ ]:
def window_decorator(window_size=10000):
    def window_calc(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            temp = []
            df = args[0]
            inspect_params = inspect.getfullargspec(func)
            for i in tqdm(range(0, df.shape[0], window_size), 
                    desc=func.__name__ + "({}, ".format(*inspect_params.args)
                          + ', '.join("{}={})".format(k, v) for k, v in kwargs.items())):
                batch = df.iloc[i: i+window_size].values
                temp.append(func(batch, *args, **kwargs))
            return pd.DataFrame(temp, columns={func.__name__})
        return wrapper
    return window_calc

In [ ]:
@window_decorator()
def w_autocorrelation(df, *args, lag=100, **kwargs):
    return tsfresh.feature_extraction.feature_calculators.autocorrelation(df, lag=lag)
@window_decorator()
def w_psd(df, *args, fs=4e6, **kwargs):
    return np.sum(scipy.signal.periodogram(df, fs=fs)[1])
@window_decorator()
def w_min(df, *args, **kwargs):
    return np.min(df)
@window_decorator()
def w_absolute_sum_of_changes(df, *args, **kwargs):
    return tsfresh.feature_extraction.feature_calculators.absolute_sum_of_changes(df)
@window_decorator()
def w_sample_entropy(df, *args, **kwargs):
    return tsfresh.feature_extraction.feature_calculators.sample_entropy(df)
@window_decorator()
def w_symmetry_looking(df, *args, r=0.1, **kwargs):
    return tsfresh.feature_extraction.feature_calculators.symmetry_looking(df, param={"r": r})